# Preprocessing of cancer data

In [1]:
#import useful libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from matplotlib.pyplot import figure
import sys
import umap.umap_ as umap
import random
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.decomposition import KernelPCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import KFold

In [2]:
#instant read in
df = pd.read_csv('../clinical_TumorCompendium_v11_PolyA_2020-04-09.tsv', sep='\t', header=0)

In [3]:
#run this cell only if you haven't created the reduced data from 58581 features to 33466 features

# #import data (beware, it takes around 11 minutes to read in and 13 minutes to write)
# df2 = pd.read_csv('../TumorCompendium_v11_PolyA_hugo_log2tpm_58581genes_2020-04-09.tsv', sep='\t', header=0)
# geneNames = pd.read_csv('../TumorCompendium_v11_gene_names_reduced.tsv', sep='\t', header=0)
# reduced_features = pd.merge(geneNames, df2, on='Gene')
# reduced_features.to_csv("TumorCompendium_v11_PolyA_hugo_log2tpm_33466genes_reduced.tsv", sep="\t")

In [4]:
#takes around 6 minutes
df2 = pd.read_csv("TumorCompendium_v11_PolyA_hugo_log2tpm_33466genes_reduced.tsv", sep="\t", header=0)

In [5]:
df

,th_sampleid,disease,age_at_dx,pedaya,gender,site_id,dataset_accession,site_donor_id,site_sampleid
0,TH03_0010_S01,acute leukemia of ambiguous lineage,11.0,"Yes, age < 30 years",female,TH03,unavailable,NaN,NaN
1,TH03_0010_S02,acute leukemia of ambiguous lineage,11.0,"Yes, age < 30 years",female,TH03,unavailable,NaN,NaN
2,THR33_1000_S01,medulloblastoma,7.0,"Yes, age < 30 years",female,THR33,EGAD00001003279,NaN,NaN
3,THR33_1001_S01,medulloblastoma,5.0,"Yes, age < 30 years",male,THR33,EGAD00001003279,NaN,NaN
4,THR33_1002_S01,medulloblastoma,5.0,"Yes, age < 30 years",female,THR33,EGAD00001003279,NaN,NaN
...,...,...,...,...,...,...,...,...,...
12742,TCGA-ZS-A9CF-02,hepatocellular carcinoma,64.0,No,male,TCGA,phs000178,TCGA-ZS-A9CF,TCGA-ZS-A9CF-02
12743,TCGA-ZS-A9CG-01,hepatocellular carcinoma,55.0,No,male,TCGA,phs000178,TCGA-ZS-A9CG,TCGA-ZS-A9CG-01
12744,TCGA-ZT-A8OM-01,thymoma,73.0,No,female,TCGA,phs000178,TCGA-ZT-A8OM,TCGA-ZT-A8OM-01
12745,TCGA-ZU-A8S4-01,cholangiocarcinoma,52.0,No,male,TCGA,phs000178,TCGA-ZU-A8S4,TCGA-ZU-A8S4-01


In [6]:
df2

,Unnamed: 0,Gene,TH40_2202_S02,TH40_2202_S03,TH27_1241_S01,TH40_2202_S01,TARGET-40-0A4I65-01A-01R,TH40_2274_S01,THR24_1965_S01,THR24_2080_S01,...,TCGA-BP-4803-01,TCGA-A8-A09K-01,TCGA-61-1907-01,TCGA-IB-7885-01,TCGA-B6-A0IA-01,TCGA-VQ-AA6F-01,TCGA-BR-8588-01,TCGA-G3-A25X-01,TCGA-24-2254-01,TCGA-DD-A115-01
0,0,A1BG,4.651913,5.557655,4.586164,4.866908,4.935460,4.177121,5.098453,5.528571,...,2.424979,4.679187,5.000447,3.129315,6.213949,1.526119,1.646188,7.107788,2.992791,9.799418
1,1,A1BG-AS1,2.536053,3.026800,3.000000,2.625270,2.464668,2.475085,2.414136,3.023255,...,1.070450,3.106042,1.659971,1.310415,2.378513,0.739941,0.918464,0.879794,1.521095,0.739941
2,2,A1CF,0.056584,0.014355,0.028569,0.000000,0.000000,0.000000,0.014355,0.000000,...,2.053139,0.000000,0.000000,0.475177,0.000000,1.056670,2.039190,2.541017,0.000000,4.890918
3,3,A2M,6.637784,5.936873,5.266412,5.775314,7.760021,4.151372,5.587065,7.100242,...,8.840647,6.531269,6.084696,8.018768,4.224161,7.372823,7.635666,10.954727,7.379781,9.240585
4,4,A2M-AS1,1.709291,0.748461,0.432959,1.555816,1.201634,1.169925,1.531069,1.111031,...,1.511009,1.560755,1.281002,0.807444,0.189164,1.214170,0.887589,1.433023,2.131002,0.713785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33461,33461,ZYG11B,3.689299,3.139142,4.324091,3.114367,3.468583,3.129283,3.193772,2.066950,...,3.472477,3.085818,2.356176,2.809431,2.104368,2.615877,2.794951,1.298743,3.656489,2.505903
33462,33462,ZYX,6.423578,6.886306,5.195741,6.333245,7.375995,5.578033,7.866970,6.561021,...,6.342670,5.776145,6.667351,7.757178,6.302183,7.035033,6.998318,6.694188,8.315331,5.487980
33463,33463,ZYXP1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
33464,33464,ZZEF1,2.731183,2.253989,3.948601,2.643856,3.311794,3.304511,3.674687,2.837943,...,3.209497,3.848036,1.422274,2.942983,1.887569,3.521059,3.539787,0.956147,4.082398,1.555865


This notebook takes the reduced data, which excludes the RNA sequenced data, and reduces it further by selecting the n most variable genes with respect to the classes present. This will be conducted with respect to each experiment to be conducted.

In [7]:
samples1 = df[df['disease']=='lung squamous cell carcinoma']['th_sampleid'] 
g1_samples = df2.loc[:,list(samples1)]

samples2 = df[df['disease']=='kidney clear cell carcinoma']['th_sampleid'] 
g2_samples = df2.loc[:,list(samples2)]

samples3 = df[df['disease']=='thyroid carcinoma']['th_sampleid'] 
g3_samples = df2.loc[:,list(samples3)]

samples4 = df[df['disease']=='head & neck squamous cell carcinoma']['th_sampleid'] 
g4_samples = df2.loc[:,list(samples4)]

samples5 = df[df['disease']=='kidney papillary cell carcinoma']['th_sampleid'] 
g5_samples = df2.loc[:,list(samples5)]

samples6 = df[df['disease']=='bladder urothelial carcinoma']['th_sampleid'] 
g6_samples = df2.loc[:,list(samples6)]

In [8]:
df_samples = df[df['disease']=='lung squamous cell carcinoma']
samples = df_samples[df_samples['gender']=='male']['th_sampleid']
pos_samples11 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['gender']=='female']['th_sampleid'] 
neg_samples11 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='kidney clear cell carcinoma']
samples = df_samples[df_samples['gender']=='male']['th_sampleid']
pos_samples12 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['gender']=='female']['th_sampleid'] 
neg_samples12 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='thyroid carcinoma']
samples = df_samples[df_samples['gender']=='male']['th_sampleid']
pos_samples13 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['gender']=='female']['th_sampleid'] 
neg_samples13 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='head & neck squamous cell carcinoma']
samples = df_samples[df_samples['gender']=='male']['th_sampleid']
pos_samples14 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['gender']=='female']['th_sampleid'] 
neg_samples14 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='kidney papillary cell carcinoma']
samples = df_samples[df_samples['gender']=='male']['th_sampleid']
pos_samples15 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['gender']=='female']['th_sampleid'] 
neg_samples15 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='bladder urothelial carcinoma']
samples = df_samples[df_samples['gender']=='male']['th_sampleid']
pos_samples16 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['gender']=='female']['th_sampleid'] 
neg_samples16 = df2.loc[:,list(samples)]

In [9]:
pos_samples3 = pd.concat([pos_samples11.T, pos_samples12.T, pos_samples13.T, pos_samples14.T, pos_samples15.T, pos_samples16.T])
neg_samples3 = pd.concat([neg_samples11.T, neg_samples12.T, neg_samples13.T, neg_samples14.T, neg_samples15.T, neg_samples16.T])

In [22]:
pos_samples3

,0,1,2,3,4,5,6,7,8,9,...,33456,33457,33458,33459,33460,33461,33462,33463,33464,33465
TCGA-18-3407-01,3.126038,1.406022,0.042784,6.955858,0.411534,4.237226,0.0,0.0,0.070525,0.000000,...,0.992862,1.815619,3.658791,0.150686,0.000000,2.659950,6.784931,0.0,2.488043,3.960680
TCGA-18-3409-01,5.136277,2.408721,0.000000,8.309251,1.448950,0.028710,0.0,0.0,0.000000,0.000000,...,1.863989,2.790758,3.666805,0.042784,0.070525,3.217259,7.046843,0.0,2.682630,4.267549
TCGA-18-3410-01,5.763677,3.008982,0.000000,5.627064,0.604182,3.848036,0.0,0.0,0.594646,0.000000,...,1.028655,1.599372,3.553413,1.632331,0.000000,2.469904,6.353141,0.0,2.592144,3.799055
TCGA-18-3412-01,2.526123,0.910803,0.000000,5.475075,0.632353,5.017501,0.0,0.0,0.000000,0.028710,...,0.632353,1.292813,2.500795,1.104410,0.000000,1.967194,6.041353,0.0,1.735531,3.142433
TCGA-18-3414-01,3.323396,1.014435,0.000000,5.807862,0.948698,3.790796,0.0,0.0,0.028710,0.042784,...,1.028655,2.077271,4.431591,0.848089,0.000000,2.587390,7.079894,0.0,1.952405,4.023285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZF-AA4X-01,2.250979,0.848089,0.400636,4.375756,0.879794,4.666793,0.0,0.0,0.856081,0.042784,...,1.286958,2.395097,4.440938,0.526184,0.000000,3.514761,6.161282,0.0,3.823762,3.851014
TCGA-ZF-AA52-01,4.026852,1.718147,0.000000,8.985347,1.063598,4.902033,0.0,0.0,0.070525,0.000000,...,0.356247,1.226565,3.377157,0.042784,0.000000,2.241900,7.322732,0.0,2.127684,3.400586
TCGA-ZF-AA53-01,5.775556,2.859995,0.000000,6.421910,0.263160,2.406042,0.0,0.0,0.163630,0.000000,...,0.322052,1.077333,2.606425,0.536153,0.000000,1.748505,8.001336,0.0,1.448950,2.698267
TCGA-ZF-AA54-01,5.373983,2.895335,0.000000,7.536286,1.516080,0.056724,0.0,0.0,0.124460,0.056724,...,0.970923,1.627658,3.853993,1.321971,0.000000,2.364634,8.028431,0.0,3.089172,4.320507


In [207]:
neg_samples3

,0,1,2,3,4,5,6,7,8,9,...,33456,33457,33458,33459,33460,33461,33462,33463,33464,33465
TCGA-18-3408-01,0.111167,0.782505,0.000000,7.352050,1.124389,1.774036,0.000000,0.0,0.028710,0.000000,...,1.292813,2.049648,3.676943,0.226636,0.000000,2.327753,3.967138,0.0,2.408721,3.596931
TCGA-18-3411-01,4.814071,2.235751,0.028710,5.413423,0.546055,0.454284,0.000000,0.0,0.070525,0.000000,...,0.722542,1.469961,2.784514,0.070525,0.000000,1.933605,5.953745,0.0,1.459497,3.721343
TCGA-21-1070-01,3.206375,2.440952,0.000000,6.297740,0.400636,3.330599,0.137632,0.0,0.028710,0.042784,...,0.790837,1.852037,3.837988,1.344905,0.000000,2.498243,6.351560,0.0,2.467282,3.316196
TCGA-21-1076-01,3.113560,2.003640,0.021551,8.397923,1.107754,4.555492,0.000000,0.0,0.201754,0.021551,...,0.929863,1.737724,3.944418,0.604173,0.000000,2.676934,7.481919,0.0,3.193006,4.267197
TCGA-21-5782-01,4.443610,1.952405,0.344950,6.378035,0.799151,3.499529,0.214245,0.0,0.070525,0.000000,...,0.879794,1.344905,3.788662,0.454284,0.485537,1.794981,6.758178,0.0,2.277988,3.060085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZF-AA51-01,3.321956,1.469961,0.000000,6.849850,0.536153,4.366715,0.000000,0.0,0.042784,0.000000,...,0.641648,1.555865,2.807373,0.124460,0.000000,2.550964,5.816899,0.0,2.003606,3.384028
TCGA-ZF-AA56-01,2.887547,1.555865,0.014498,7.014690,0.422330,4.917404,0.000000,0.0,0.163630,0.000000,...,0.660012,1.682619,3.392350,0.056724,0.000000,2.608850,7.416164,0.0,2.490592,3.531105
TCGA-ZF-AA58-01,2.981866,1.914598,0.000000,6.505453,0.464783,0.028710,0.000000,0.0,0.097748,0.000000,...,0.933637,1.778215,3.393707,1.618267,0.000000,2.560759,8.179680,0.0,2.408721,3.404661
TCGA-ZF-AA5H-01,1.794981,0.485537,0.000000,5.870156,0.356247,2.336329,0.000000,0.0,0.028710,0.000000,...,0.565692,1.500832,2.794951,0.042784,0.000000,2.548476,8.092092,0.0,2.726830,4.252478


In [32]:
print(pos_samples3.iloc[:,0])
print()
print(neg_samples3.iloc[:,0])

TCGA-18-3407-01    3.126038
TCGA-18-3409-01    5.136277
TCGA-18-3410-01    5.763677
TCGA-18-3412-01    2.526123
TCGA-18-3414-01    3.323396
                     ...   
TCGA-ZF-AA4X-01    2.250979
TCGA-ZF-AA52-01    4.026852
TCGA-ZF-AA53-01    5.775556
TCGA-ZF-AA54-01    5.373983
TCGA-ZF-AA5P-01    2.604084
Name: 0, Length: 1755, dtype: float64

TCGA-18-3408-01    0.111167
TCGA-18-3411-01    4.814071
TCGA-21-1070-01    3.206375
TCGA-21-1076-01    3.113560
TCGA-21-5782-01    4.443610
                     ...   
TCGA-ZF-AA51-01    3.321956
TCGA-ZF-AA56-01    2.887547
TCGA-ZF-AA58-01    2.981866
TCGA-ZF-AA5H-01    1.794981
TCGA-ZF-AA5N-01    1.176404
Name: 0, Length: 1007, dtype: float64


In [33]:
print(pos_samples3.iloc[:,0].mean())
print()
print(neg_samples3.iloc[:,0].mean())

3.2360506012988264

3.605595164811572


In [37]:
np.abs(pos_samples3.iloc[:,0].mean() - neg_samples3.iloc[:,0].mean())

0.36954456351274567

In [35]:
np.minimum(pos_samples3.iloc[:,0].mean(), neg_samples3.iloc[:,0].mean())

3.2360506012988264

In [34]:
#yields relative difference for the gene at index 0
np.abs(pos_samples3.iloc[:,0].mean() - neg_samples3.iloc[:,0].mean()) / np.minimum(pos_samples3.iloc[:,0].mean(), neg_samples3.iloc[:,0].mean())

0.11419616348533755

In [36]:
#yields relative difference for the gene at index 0
np.abs(pos_samples3.iloc[:,1].mean() - neg_samples3.iloc[:,1].mean()) / np.minimum(pos_samples3.iloc[:,1].mean(), neg_samples3.iloc[:,1].mean())

0.17489612282324865

In [95]:
pos_samples3.stack().mean()

1.6142461833079482

In [96]:
neg_samples3.stack().mean()

1.618173875998735

In [100]:
dif_all_genes = np.abs(pos_samples3.stack().mean() - neg_samples3.stack().mean())

In [88]:
#average relative difference across all genes
relative_dif_all_genes = np.abs(pos_samples3.stack().mean() - neg_samples3.stack().mean()) / np.minimum(pos_samples3.stack().mean(), neg_samples3.stack().mean())
relative_dif_all_genes

0.002433143551089606

In [121]:
featureSize = df2.shape[0]
featureSize

33466

In [124]:
pos_samples3.shape

(1755, 33466)

In [123]:
neg_samples3.shape

(1007, 33466)

In [79]:
relativeDifferences = np.zeros(featureSize)

for i in range(0,featureSize):
    
    relativeDifferences[i] = np.abs(pos_samples3.iloc[:,i].mean() - 
                                    neg_samples3.iloc[:,i].mean()) / np.minimum(pos_samples3.iloc[:,i].mean(), neg_samples3.iloc[:,i].mean())
    if relativeDifferences[i] == np.inf:
        relativeDifferences[i] = pos_samples3.iloc[:,i].mean() - neg_samples3.iloc[:,i].mean()
    

<ipython-input-79-f85490e9d31b>:5: RuntimeWarning: divide by zero encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples3.iloc[:,i].mean() -
<ipython-input-79-f85490e9d31b>:5: RuntimeWarning: invalid value encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples3.iloc[:,i].mean() -


In [61]:
pos_samples3.iloc[:,33463].mean()

0.0018886991375897434

In [62]:
neg_samples3.iloc[:,33463].mean()

0.0

In [60]:
np.abs(pos_samples3.iloc[:,33463].mean() - neg_samples3.iloc[:,33463].mean()) / np.minimum(pos_samples3.iloc[:,33463].mean(), neg_samples3.iloc[:,33463].mean())

<ipython-input-60-07edf7e18094>:1: RuntimeWarning: divide by zero encountered in double_scalars
  np.abs(pos_samples3.iloc[:,33463].mean() - neg_samples3.iloc[:,33463].mean()) / np.minimum(pos_samples3.iloc[:,33463].mean(), neg_samples3.iloc[:,33463].mean())


inf

In [70]:
pos_samples3.iloc[:,1363].mean()

0.0

In [71]:
neg_samples3.iloc[:,1363].mean()

0.0

In [69]:
np.abs(pos_samples3.iloc[:,1363].mean() - neg_samples3.iloc[:,1363].mean()) / np.minimum(pos_samples3.iloc[:,1363].mean(), neg_samples3.iloc[:,1363].mean())

<ipython-input-69-cb428110d084>:1: RuntimeWarning: invalid value encountered in double_scalars
  np.abs(pos_samples3.iloc[:,1363].mean() - neg_samples3.iloc[:,1363].mean()) / np.minimum(pos_samples3.iloc[:,1363].mean(), neg_samples3.iloc[:,1363].mean())


nan

In [63]:
relativeDifferences

[0.11419616 0.17489612 0.02352373 ...        inf 0.02319698 0.02303982]


In [80]:
np.sum(np.isnan(relativeDifferences))

3606

In [81]:
np.sum(np.isinf(relativeDifferences))

0

nan come from 0/0, so those can be scrapped, inf comes from dividing a value by 0, so handling those might differ (use diff).

In [68]:
for i,k in enumerate(relativeDifferences):
    print(i, ": ", k)

0 :  0.11419616348533755
1 :  0.17489612282324865
2 :  0.023523728238972256
3 :  0.0323684801772967
4 :  0.03862559579156642
5 :  0.566472428489739
6 :  0.026127476114192878
7 :  0.09125531982684636
8 :  0.07308283961831157
9 :  0.03482615768933214
10 :  0.11458066052901882
11 :  0.12929351153476654
12 :  0.011195451979488175
13 :  0.028227964228392296
14 :  0.3793429739018513
15 :  0.7324258651665839
16 :  0.9567828426662712
17 :  1.2665791839001197
18 :  0.1994401285862523
19 :  0.19480091581547462
20 :  0.2809945588109076
21 :  0.037358151672698965
22 :  0.01924303998947246
23 :  0.026764232100084002
24 :  0.03285791572257093
25 :  0.009937859084634522
26 :  0.011763226846948088
27 :  0.0008807770416769661
28 :  0.020855200006569923
29 :  0.018216777058839298
30 :  0.005227573623189063
31 :  0.020976425795524483
32 :  0.019921517236364702
33 :  0.016525425945248515
34 :  0.5849897791222635
35 :  0.029181075268860497
36 :  0.2098452783027383
37 :  0.03149378221900937
38 :  0.25710193

765 :  0.5875087660397409
766 :  0.005591096322784585
767 :  0.03693607704787425
768 :  0.0017922369990494417
769 :  0.0686680470035096
770 :  0.02973638919952015
771 :  0.012326240651580453
772 :  0.04141642638311468
773 :  0.02147022893666561
774 :  0.0662142067102213
775 :  0.051916483388862185
776 :  0.011473814941245794
777 :  0.23938170189970936
778 :  0.5447133930274285
779 :  0.04665827343131233
780 :  0.2839546267820336
781 :  0.4867731889954242
782 :  0.004028699239231946
783 :  0.6267769210403488
784 :  0.3264707241897302
785 :  0.41073566086687396
786 :  0.038393984393967566
787 :  0.02360498215000493
788 :  0.2673268722011628
789 :  0.18490672326607133
790 :  0.00035224289654833005
791 :  0.1930192197042834
792 :  0.06314968492773383
793 :  0.05051900294920843
794 :  0.3190077555934364
795 :  0.004250077407556852
796 :  0.07836310008353724
797 :  0.6328217823212652
798 :  0.432541080254687
799 :  0.05859335148721327
800 :  0.038911022181196124
801 :  0.064848637484841
802 

1501 :  0.02193485182300607
1502 :  0.14631326920891596
1503 :  0.0006130221689659133
1504 :  0.051803903953930745
1505 :  0.05519178964949579
1506 :  0.04006565578324097
1507 :  0.12990808276116833
1508 :  0.23262406653809004
1509 :  0.17679824935202193
1510 :  0.07526580642734877
1511 :  0.6596962168998848
1512 :  0.4586967953110408
1513 :  1.4250336199875897
1514 :  0.6439503879161557
1515 :  0.07316459572664508
1516 :  0.008731310714256438
1517 :  0.00461035293116594
1518 :  0.2739447442753264
1519 :  0.060223187319186695
1520 :  0.698959678855135
1521 :  0.0266147249149864
1522 :  0.016238948032246903
1523 :  0.07528804125995137
1524 :  0.8258409797198981
1525 :  0.015532308045305029
1526 :  0.10213802981737777
1527 :  0.03031536177931864
1528 :  0.054106762792816
1529 :  0.07562250016884965
1530 :  0.27633235827142644
1531 :  0.3466517575835116
1532 :  0.03023346071697148
1533 :  0.03319708700752723
1534 :  0.03279815390190856
1535 :  0.29232849294615604
1536 :  0.029197221551583

2298 :  0.10516678427847448
2299 :  0.07914932263999035
2300 :  0.4361607511528285
2301 :  0.2854045684340727
2302 :  0.3939489894092863
2303 :  0.31956423084687857
2304 :  0.5113473549795915
2305 :  0.00037609650606512836
2306 :  0.15279732899387752
2307 :  0.6056651624704543
2308 :  0.6908022509157337
2309 :  0.022400648314153827
2310 :  0.5077657530746376
2311 :  0.15658073795962021
2312 :  0.042689045222389234
2313 :  0.09370427020305248
2314 :  0.00667177625850981
2315 :  0.08325250421546468
2316 :  0.06506307529380662
2317 :  0.13392492577586837
2318 :  0.700419067311341
2319 :  0.05986710101074739
2320 :  0.026640749202039578
2321 :  0.19165828459083503
2322 :  0.037689370145497156
2323 :  0.0937609420268799
2324 :  0.03075019342456924
2325 :  0.23492369649220404
2326 :  0.7956689630851155
2327 :  0.32802442470203874
2328 :  0.06856085679809253
2329 :  0.001977991382495911
2330 :  0.6883399086435071
2331 :  0.01666613107115891
2332 :  1.2235107725926315
2333 :  0.024001022835619

3083 :  0.0753453891329228
3084 :  0.10995680800879873
3085 :  0.09921198254765405
3086 :  0.005273132118069826
3087 :  0.03446899084916924
3088 :  0.005193435340495582
3089 :  0.06799862495071538
3090 :  0.026732230397831645
3091 :  0.07699836893079744
3092 :  0.01581638099934593
3093 :  2.7825380287441166
3094 :  0.06851695855317678
3095 :  0.008391268901659737
3096 :  0.19040294495858698
3097 :  0.4428011982606677
3098 :  0.5810031070113149
3099 :  0.35692511125019133
3100 :  0.024935253248280612
3101 :  0.4009198218281073
3102 :  0.12211405559507905
3103 :  0.24894338103533148
3104 :  0.27458201081121963
3105 :  0.15419021164518718
3106 :  0.24329696582518745
3107 :  0.2437334909531044
3108 :  0.0748196403097971
3109 :  0.01018927302690289
3110 :  0.024820610066362676
3111 :  0.23025335889116083
3112 :  0.020782146423602127
3113 :  0.007648734288887216
3114 :  0.08230921277569472
3115 :  0.2083765739430695
3116 :  0.11959582355620302
3117 :  0.226548387823425
3118 :  0.216579058592

...

29872 :  0.032600260156862175
29873 :  4.0723838221981256e-05
29874 :  0.18241519880599488
29875 :  0.04212409719918305
29876 :  0.05200369704798207
29877 :  0.19531825164135355
29878 :  0.07283526198417337
29879 :  0.621808112582672
29880 :  0.38744651789107487
29881 :  1.0624202920844745
29882 :  1.0373864718955528
29883 :  0.032948060604988054
29884 :  0.002150049570258851
29885 :  0.04547032689724762
29886 :  0.010656684110714054
29887 :  0.031609474541032696
29888 :  0.08787447594879395
29889 :  0.2738526005408008
29890 :  0.15753373939943738
29891 :  0.3716635607245326
29892 :  0.9963848452504044
29893 :  0.25787353824587095
29894 :  0.2262892254125257
29895 :  0.18633428393236673
29896 :  0.08850156989517813
29897 :  0.006197033770017729
29898 :  0.0284106753262175
29899 :  0.019223909140525963
29900 :  0.023128861160841646
29901 :  0.020021335114813004
29902 :  0.04404740338943625
29903 :  0.061076227293515084
29904 :  0.25365778314210474
29905 :  0.009896706077175448
29906 :  

30760 :  nan
30761 :  nan
30762 :  0.1581199738914185
30763 :  0.4360200272408193
30764 :  0.0751033579793328
30765 :  0.023697556701630824
30766 :  0.032230280906143015
30767 :  0.0189710596011983
30768 :  0.05899121115208678
30769 :  0.019880983375374226
30770 :  0.03286011807024845
30771 :  0.05052174912063826
30772 :  0.5944867679715095
30773 :  0.5736757384780441
30774 :  0.2364968627625407
30775 :  0.1720171901884546
30776 :  0.06159509549752645
30777 :  0.048404786890866094
30778 :  0.5808417739369289
30779 :  0.030868456852244878
30780 :  0.0065997870319192235
30781 :  0.01623919587266859
30782 :  nan
30783 :  nan
30784 :  nan
30785 :  nan
30786 :  nan
30787 :  0.12378328495127612
30788 :  0.00042508412744114216
30789 :  0.08604291871136081
30790 :  0.0010755337930718541
30791 :  0.04241345516498179
30792 :  0.08600559524248307
30793 :  0.0523092323995316
30794 :  0.06182916881550106
30795 :  0.13185838494791552
30796 :  0.10043024618062982
30797 :  0.02795900496170865
30798 : 

31403 :  0.10188657337695735
31404 :  0.09987416820051934
31405 :  0.0002412864400762106
31406 :  0.015617583490990952
31407 :  0.28936255917128406
31408 :  0.0528244416688245
31409 :  0.1537313812800793
31410 :  0.048218769000218174
31411 :  1.361649446468763
31412 :  0.019693423646375965
31413 :  0.006079352601407203
31414 :  0.006154313904088972
31415 :  0.008660788222343766
31416 :  0.05830870045098297
31417 :  0.10767776004848251
31418 :  1.0435584518088377
31419 :  0.045022471688103055
31420 :  0.11700619160965603
31421 :  0.007909481107481324
31422 :  0.30509624664992874
31423 :  0.05324077536787672
31424 :  0.2616365360055796
31425 :  0.006898190292183519
31426 :  0.020617928216064757
31427 :  0.101545010753322
31428 :  0.1931503837579566
31429 :  0.4918919771114134
31430 :  0.12239282726316698
31431 :  0.04865036250244349
31432 :  0.015544305213697245
31433 :  0.009081193963889225
31434 :  0.15460543468591761
31435 :  0.004726088446716928
31436 :  0.043817796021035055
31437 : 

32341 :  0.043300124746542816
32342 :  0.013251595902046642
32343 :  0.011956592371675623
32344 :  0.0222365536688064
32345 :  0.024109750582477652
32346 :  0.21164492503806165
32347 :  0.002995211933743743
32348 :  0.020714244075382887
32349 :  0.35081587766859484
32350 :  0.025764332059947534
32351 :  0.5610019970077708
32352 :  0.5456290326704168
32353 :  0.6745430714735894
32354 :  0.7043359951260186
32355 :  1.7486964200884672
32356 :  0.020654342416533064
32357 :  0.10124240793924634
32358 :  0.24565439937550584
32359 :  0.03409696563171993
32360 :  0.08181387645850927
32361 :  0.29016322298919867
32362 :  0.229760852593821
32363 :  nan
32364 :  0.03588015705899697
32365 :  0.5988910925427329
32366 :  0.361303020703275
32367 :  0.4347704371470718
32368 :  1.0327261602988036
32369 :  28.520753445577654
32370 :  0.05536065408781205
32371 :  0.5625193917937262
32372 :  0.052374185783270855
32373 :  0.19940872352626896
32374 :  0.28951322114832384
32375 :  0.1478214364353638
32376 : 

33037 :  0.22288621652479404
33038 :  0.04280463272973131
33039 :  0.1100372771619006
33040 :  0.029935943492422047
33041 :  0.1140537395438627
33042 :  0.15154211835925346
33043 :  0.04390699567060087
33044 :  0.029102844861507014
33045 :  0.02644978158913711
33046 :  0.10932419282018536
33047 :  0.006334920005021049
33048 :  0.054010542698462526
33049 :  0.014020485140008188
33050 :  0.07211456054934173
33051 :  0.014829930222201092
33052 :  0.011771402292870523
33053 :  0.015981218952546224
33054 :  0.03816717739598549
33055 :  0.041078393015504255
33056 :  0.019421630592955933
33057 :  0.05130259356714968
33058 :  0.10900967762345365
33059 :  0.4635237062334691
33060 :  0.7202313324656759
33061 :  0.5835606068629227
33062 :  nan
33063 :  0.10481579392225317
33064 :  0.05169509465248197
33065 :  0.07041480418836084
33066 :  0.026525286709012007
33067 :  0.03937916454087369
33068 :  0.08597654000196582
33069 :  0.17205619493178984
33070 :  0.079321635577998
33071 :  0.102006033579107

In [87]:
x = np.nan
if np.isnan(x):
    print("Bitch")

Bitch


In [104]:
indicesToKeep = []
for i in range(0, featureSize):
    if not np.isnan(relativeDifferences[i]):
        if relativeDifferences[i] > relative_dif_all_genes:
            indicesToKeep.append(i)

In [106]:
indx = np.array(indicesToKeep)
indx.shape

(28562,)

In [107]:
n_most_variable_indices = np.argsort(relativeDifferences)[::-1][:8000]

In [110]:
n_most_variable_indices

array([24769, 24103, 24089, ..., 30215,  9427,  2909], dtype=int64)

In [128]:
ind = np.sort(n_most_variable_indices)
ind

array([    5,    15,    16, ..., 33442, 33448, 33460], dtype=int64)

In [116]:
test = pos_samples3.iloc[:,ind]
test

,5,15,16,17,34,53,57,58,68,69,...,33387,33388,33400,33402,33404,33410,33419,33442,33448,33460
TCGA-18-3407-01,4.237226,1.594617,0.097748,0.070525,2.201667,0.189164,0.000000,0.028710,0.070525,0.014498,...,0.0,0.042784,0.0,0.014498,0.000000,0.00000,0.000000,0.0,0.0,0.000000
TCGA-18-3409-01,0.028710,2.378513,0.604182,1.622993,1.459497,0.111167,0.000000,0.097748,0.189164,0.333534,...,0.0,0.546055,0.0,0.042784,0.000000,0.00000,0.028710,0.0,0.0,0.070525
TCGA-18-3410-01,3.848036,0.286996,0.014498,0.111167,2.039190,0.687133,0.000000,0.028710,0.014498,0.000000,...,0.0,0.176446,0.0,0.028710,0.000000,0.38968,0.000000,0.0,0.0,0.000000
TCGA-18-3412-01,5.017501,4.467948,3.650781,0.000000,1.541050,1.575342,0.000000,0.028710,0.344950,0.000000,...,0.0,0.000000,0.0,0.028710,0.000000,0.00000,0.070525,0.0,0.0,0.000000
TCGA-18-3414-01,3.790796,2.669048,2.752768,0.807444,1.604135,1.618267,0.000000,0.594646,0.400636,0.028710,...,0.0,0.000000,0.0,0.014498,0.000000,0.00000,0.201757,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZF-AA4X-01,4.666793,1.546046,0.056724,0.000000,3.129315,0.000000,0.000000,0.070525,0.322052,0.000000,...,0.0,0.000000,0.0,0.000000,0.084203,0.00000,2.639266,0.0,0.0,0.000000
TCGA-ZF-AA52-01,4.902033,0.000000,0.000000,0.000000,1.490630,0.111167,0.000000,0.000000,0.056724,0.000000,...,0.0,0.000000,0.0,0.042784,0.000000,0.00000,1.189114,0.0,0.0,0.000000
TCGA-ZF-AA53-01,2.406042,0.000000,0.000000,0.000000,3.028621,0.000000,0.000000,0.000000,0.176446,0.000000,...,0.0,0.111167,0.0,0.014498,0.000000,0.00000,0.189164,0.0,0.0,0.000000
TCGA-ZF-AA54-01,0.056724,0.150686,0.000000,0.000000,0.310455,0.000000,0.028710,0.000000,0.111167,0.000000,...,0.0,1.550918,0.0,0.070525,0.000000,0.00000,0.000000,0.0,0.0,0.000000


In [117]:
test.shape

(1755, 8000)

In [118]:
# Example for saving a dataframe to a csv file: df.to_csv("education_salary.tsv", sep="\t")

In [95]:
#method to select the n most variable features for the particular experiment

def write_n_most_variable(experimentName, pos_samples, neg_samples, n):
    featureSize = pos_samples.shape[1]
    relativeDifferences = np.zeros(featureSize)

    for i in range(0,featureSize):

        relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() - 
                                        neg_samples.iloc[:,i].mean()) / np.minimum(pos_samples.iloc[:,i].mean(), 
                                                                                   neg_samples.iloc[:,i].mean())
        if relativeDifferences[i] == np.inf:
            relativeDifferences[i] = pos_samples.iloc[:,i].mean() - neg_samples.iloc[:,i].mean()
        if np.isnan(relativeDifferences[i]):
            relativeDifferences[i] = 0.0
    
    n_most_variable_indices = np.argsort(relativeDifferences)[::-1][:n]
    ind = np.sort(n_most_variable_indices)
    df_to_write_pos = pos_samples.iloc[:,ind]
    df_to_write_neg = neg_samples.iloc[:,ind]
    df_to_write_pos.to_csv(experimentName + "_pos_samples.tsv", sep='\t')
    df_to_write_neg.to_csv(experimentName + "_neg_samples.tsv", sep='\t')
    return


In [40]:
write_n_most_variable("male_v_female_carcinomas", pos_samples3, neg_samples3, 8000)

<ipython-input-26-85dc9684092f>:9: RuntimeWarning: divide by zero encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -
<ipython-input-26-85dc9684092f>:9: RuntimeWarning: invalid value encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -


In [41]:
df_test1 = pd.read_csv('male_v_female_carcinomas_pos_samples.tsv', sep='\t', header=0)
df_test2 = pd.read_csv('male_v_female_carcinomas_neg_samples.tsv', sep='\t', header=0)
#df = df.iloc[: , 1:]
df_test1.rows = df_test1.iloc[:,0]
df_test2.rows = df_test2.iloc[:,0]

<ipython-input-41-b4382e644621>:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_test1.rows = df_test1.iloc[:,0]
<ipython-input-41-b4382e644621>:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_test2.rows = df_test2.iloc[:,0]


In [42]:
df_test1

,Unnamed: 0,5,15,16,17,34,53,57,58,68,...,33387,33388,33400,33402,33404,33410,33419,33442,33448,33460
0,TCGA-18-3407-01,4.237226,1.594617,0.097748,0.070525,2.201667,0.189164,0.000000,0.028710,0.070525,...,0.0,0.042784,0.0,0.014498,0.000000,0.00000,0.000000,0.0,0.0,0.000000
1,TCGA-18-3409-01,0.028710,2.378513,0.604182,1.622993,1.459497,0.111167,0.000000,0.097748,0.189164,...,0.0,0.546055,0.0,0.042784,0.000000,0.00000,0.028710,0.0,0.0,0.070525
2,TCGA-18-3410-01,3.848036,0.286996,0.014498,0.111167,2.039190,0.687133,0.000000,0.028710,0.014498,...,0.0,0.176446,0.0,0.028710,0.000000,0.38968,0.000000,0.0,0.0,0.000000
3,TCGA-18-3412-01,5.017501,4.467948,3.650781,0.000000,1.541050,1.575342,0.000000,0.028710,0.344950,...,0.0,0.000000,0.0,0.028710,0.000000,0.00000,0.070525,0.0,0.0,0.000000
4,TCGA-18-3414-01,3.790796,2.669048,2.752768,0.807444,1.604135,1.618267,0.000000,0.594646,0.400636,...,0.0,0.000000,0.0,0.014498,0.000000,0.00000,0.201757,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1750,TCGA-ZF-AA4X-01,4.666793,1.546046,0.056724,0.000000,3.129315,0.000000,0.000000,0.070525,0.322052,...,0.0,0.000000,0.0,0.000000,0.084203,0.00000,2.639266,0.0,0.0,0.000000
1751,TCGA-ZF-AA52-01,4.902033,0.000000,0.000000,0.000000,1.490630,0.111167,0.000000,0.000000,0.056724,...,0.0,0.000000,0.0,0.042784,0.000000,0.00000,1.189114,0.0,0.0,0.000000
1752,TCGA-ZF-AA53-01,2.406042,0.000000,0.000000,0.000000,3.028621,0.000000,0.000000,0.000000,0.176446,...,0.0,0.111167,0.0,0.014498,0.000000,0.00000,0.189164,0.0,0.0,0.000000
1753,TCGA-ZF-AA54-01,0.056724,0.150686,0.000000,0.000000,0.310455,0.000000,0.028710,0.000000,0.111167,...,0.0,1.550918,0.0,0.070525,0.000000,0.00000,0.000000,0.0,0.0,0.000000


In [43]:
df_test4 = df_test1.set_index(df_test1.iloc[:,0])
df_test4 = df_test4.iloc[:, 1:]
df_test4

,5,15,16,17,34,53,57,58,68,69,...,33387,33388,33400,33402,33404,33410,33419,33442,33448,33460
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
TCGA-18-3407-01,4.237226,1.594617,0.097748,0.070525,2.201667,0.189164,0.000000,0.028710,0.070525,0.014498,...,0.0,0.042784,0.0,0.014498,0.000000,0.00000,0.000000,0.0,0.0,0.000000
TCGA-18-3409-01,0.028710,2.378513,0.604182,1.622993,1.459497,0.111167,0.000000,0.097748,0.189164,0.333534,...,0.0,0.546055,0.0,0.042784,0.000000,0.00000,0.028710,0.0,0.0,0.070525
TCGA-18-3410-01,3.848036,0.286996,0.014498,0.111167,2.039190,0.687133,0.000000,0.028710,0.014498,0.000000,...,0.0,0.176446,0.0,0.028710,0.000000,0.38968,0.000000,0.0,0.0,0.000000
TCGA-18-3412-01,5.017501,4.467948,3.650781,0.000000,1.541050,1.575342,0.000000,0.028710,0.344950,0.000000,...,0.0,0.000000,0.0,0.028710,0.000000,0.00000,0.070525,0.0,0.0,0.000000
TCGA-18-3414-01,3.790796,2.669048,2.752768,0.807444,1.604135,1.618267,0.000000,0.594646,0.400636,0.028710,...,0.0,0.000000,0.0,0.014498,0.000000,0.00000,0.201757,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZF-AA4X-01,4.666793,1.546046,0.056724,0.000000,3.129315,0.000000,0.000000,0.070525,0.322052,0.000000,...,0.0,0.000000,0.0,0.000000,0.084203,0.00000,2.639266,0.0,0.0,0.000000
TCGA-ZF-AA52-01,4.902033,0.000000,0.000000,0.000000,1.490630,0.111167,0.000000,0.000000,0.056724,0.000000,...,0.0,0.000000,0.0,0.042784,0.000000,0.00000,1.189114,0.0,0.0,0.000000
TCGA-ZF-AA53-01,2.406042,0.000000,0.000000,0.000000,3.028621,0.000000,0.000000,0.000000,0.176446,0.000000,...,0.0,0.111167,0.0,0.014498,0.000000,0.00000,0.189164,0.0,0.0,0.000000


In [44]:
df_test2

,Unnamed: 0,5,15,16,17,34,53,57,58,68,...,33387,33388,33400,33402,33404,33410,33419,33442,33448,33460
0,TCGA-18-3408-01,1.774036,3.295700,0.765625,0.0,1.021528,1.546046,0.000000,0.056724,0.298782,...,0.00000,0.042784,0.0,0.000000,0.000000,0.000000,0.000000,0.042784,0.014498,0.000000
1,TCGA-18-3411-01,0.454284,0.903095,0.344950,0.0,1.163561,0.150686,0.000000,0.056724,0.137632,...,0.00000,0.137632,0.0,0.000000,0.000000,0.000000,0.042784,0.000000,0.000000,0.000000
2,TCGA-21-1070-01,3.330599,0.895387,0.000000,0.0,0.815671,0.097748,0.604182,0.378614,0.150686,...,0.00000,0.042784,0.0,0.028710,0.000000,0.014498,0.124460,0.000000,0.000000,0.000000
3,TCGA-21-1076-01,4.555492,2.484144,1.606472,0.0,1.467329,0.130993,0.000000,0.014426,0.124463,...,0.00000,0.049702,0.0,0.014426,0.000000,0.000000,0.292903,0.007267,0.000000,0.000000
4,TCGA-21-5782-01,3.499529,3.403303,0.443721,0.0,3.124356,1.475145,0.226636,0.214245,0.251076,...,0.26316,0.713785,0.0,0.411534,0.310455,0.150686,0.310455,0.495795,0.378614,0.485537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,TCGA-ZF-AA51-01,4.366715,0.632353,0.000000,0.0,3.565586,0.000000,0.014498,0.000000,0.201757,...,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.056724,0.000000,0.000000,0.000000
1003,TCGA-ZF-AA56-01,4.917404,1.097679,0.275124,0.0,1.967194,0.056724,0.000000,0.070525,0.238915,...,0.00000,0.000000,0.0,0.014498,0.000000,0.042784,0.214245,0.000000,0.000000,0.000000
1004,TCGA-ZF-AA58-01,0.028710,0.000000,0.000000,0.0,0.739941,0.000000,0.000000,0.000000,0.214245,...,0.00000,0.028710,0.0,0.014498,0.028710,0.028710,0.623024,0.000000,0.000000,0.000000
1005,TCGA-ZF-AA5H-01,2.336329,0.000000,0.000000,0.0,0.310455,0.378614,0.000000,0.042784,0.189164,...,0.00000,0.014498,0.0,0.097748,0.000000,0.042784,0.042784,0.000000,0.000000,0.000000


In [45]:
df_test3 = df_test2.set_index(df_test2.iloc[:,0])
df_test3 = df_test3.iloc[:, 1:]
df_test3

,5,15,16,17,34,53,57,58,68,69,...,33387,33388,33400,33402,33404,33410,33419,33442,33448,33460
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
TCGA-18-3408-01,1.774036,3.295700,0.765625,0.0,1.021528,1.546046,0.000000,0.056724,0.298782,0.028710,...,0.00000,0.042784,0.0,0.000000,0.000000,0.000000,0.000000,0.042784,0.014498,0.000000
TCGA-18-3411-01,0.454284,0.903095,0.344950,0.0,1.163561,0.150686,0.000000,0.056724,0.137632,0.000000,...,0.00000,0.137632,0.0,0.000000,0.000000,0.000000,0.042784,0.000000,0.000000,0.000000
TCGA-21-1070-01,3.330599,0.895387,0.000000,0.0,0.815671,0.097748,0.604182,0.378614,0.150686,0.014498,...,0.00000,0.042784,0.0,0.028710,0.000000,0.014498,0.124460,0.000000,0.000000,0.000000
TCGA-21-1076-01,4.555492,2.484144,1.606472,0.0,1.467329,0.130993,0.000000,0.014426,0.124463,0.007267,...,0.00000,0.049702,0.0,0.014426,0.000000,0.000000,0.292903,0.007267,0.000000,0.000000
TCGA-21-5782-01,3.499529,3.403303,0.443721,0.0,3.124356,1.475145,0.226636,0.214245,0.251076,0.516128,...,0.26316,0.713785,0.0,0.411534,0.310455,0.150686,0.310455,0.495795,0.378614,0.485537
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZF-AA51-01,4.366715,0.632353,0.000000,0.0,3.565586,0.000000,0.014498,0.000000,0.201757,0.000000,...,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.056724,0.000000,0.000000,0.000000
TCGA-ZF-AA56-01,4.917404,1.097679,0.275124,0.0,1.967194,0.056724,0.000000,0.070525,0.238915,0.000000,...,0.00000,0.000000,0.0,0.014498,0.000000,0.042784,0.214245,0.000000,0.000000,0.000000
TCGA-ZF-AA58-01,0.028710,0.000000,0.000000,0.0,0.739941,0.000000,0.000000,0.000000,0.214245,0.000000,...,0.00000,0.028710,0.0,0.014498,0.028710,0.028710,0.623024,0.000000,0.000000,0.000000


In [46]:
x = pd.concat([df_test1, df_test2])
x.iloc[0].shape

(8001,)

In [47]:
x.shape

(2762, 8001)

In [10]:
df_samples = df[df['disease']=='lung adenocarcinoma']
samples = df_samples[df_samples['gender']=='male']['th_sampleid']
pos_samples17 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['gender']=='female']['th_sampleid'] 
neg_samples17 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='stomach adenocarcinoma']
samples = df_samples[df_samples['gender']=='male']['th_sampleid']
pos_samples18 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['gender']=='female']['th_sampleid'] 
neg_samples18 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='prostate adenocarcinoma']
samples = df_samples[df_samples['gender']=='male']['th_sampleid']
pos_samples19 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['gender']=='female']['th_sampleid'] 
neg_samples19 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='colon adenocarcinoma']
samples = df_samples[df_samples['gender']=='male']['th_sampleid']
pos_samples20 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['gender']=='female']['th_sampleid'] 
neg_samples20 = df2.loc[:,list(samples)]

In [11]:
pos_samples4 = pd.concat([pos_samples17.T, pos_samples18.T, pos_samples19.T, pos_samples20.T])
neg_samples4 = pd.concat([neg_samples17.T, neg_samples18.T, neg_samples19.T, neg_samples20.T])

In [223]:
write_n_most_variable("male_v_female_adenocarcinomas", pos_samples4, neg_samples4, 8000)

<ipython-input-219-60ddce655a62>:9: RuntimeWarning: divide by zero encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -
<ipython-input-219-60ddce655a62>:9: RuntimeWarning: invalid value encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -


In [12]:
df_samples = df[df['disease']=='lung squamous cell carcinoma']
samples = df_samples[df_samples['pedaya']=='Yes, age < 30 years']['th_sampleid']
pos_samples21 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['pedaya']=='No']['th_sampleid'] 
neg_samples21 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='kidney clear cell carcinoma']
samples = df_samples[df_samples['pedaya']=='Yes, age < 30 years']['th_sampleid']
pos_samples22 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['pedaya']=='No']['th_sampleid'] 
neg_samples22 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='thyroid carcinoma']
samples = df_samples[df_samples['pedaya']=='Yes, age < 30 years']['th_sampleid']
pos_samples23 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['pedaya']=='No']['th_sampleid'] 
neg_samples23 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='head & neck squamous cell carcinoma']
samples = df_samples[df_samples['pedaya']=='Yes, age < 30 years']['th_sampleid']
pos_samples24 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['pedaya']=='No']['th_sampleid'] 
neg_samples24 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='kidney papillary cell carcinoma']
samples = df_samples[df_samples['pedaya']=='Yes, age < 30 years']['th_sampleid']
pos_samples25 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['pedaya']=='No']['th_sampleid'] 
neg_samples25 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='bladder urothelial carcinoma']
samples = df_samples[df_samples['pedaya']=='Yes, age < 30 years']['th_sampleid']
pos_samples26 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['pedaya']=='No']['th_sampleid'] 
neg_samples26 = df2.loc[:,list(samples)]

In [13]:
pos_samples5 = pd.concat([pos_samples21.T, pos_samples22.T, pos_samples23.T, pos_samples24.T, pos_samples25.T, pos_samples26.T])
neg_samples5 = pd.concat([neg_samples21.T, neg_samples22.T, neg_samples23.T, neg_samples24.T, neg_samples25.T, neg_samples26.T])

In [226]:
write_n_most_variable("ped_v_nonped_carcinomas", pos_samples5, neg_samples5, 8000)

<ipython-input-219-60ddce655a62>:9: RuntimeWarning: divide by zero encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -
<ipython-input-219-60ddce655a62>:9: RuntimeWarning: invalid value encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -


In [14]:
df_samples = df[df['disease']=='lung adenocarcinoma']
samples = df_samples[df_samples['pedaya']=='Yes, age < 30 years']['th_sampleid']
pos_samples27 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['pedaya']=='No']['th_sampleid'] 
neg_samples27 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='stomach adenocarcinoma']
samples = df_samples[df_samples['pedaya']=='Yes, age < 30 years']['th_sampleid']
pos_samples28 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['pedaya']=='No']['th_sampleid'] 
neg_samples28 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='prostate adenocarcinoma']
samples = df_samples[df_samples['pedaya']=='Yes, age < 30 years']['th_sampleid']
pos_samples29 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['pedaya']=='No']['th_sampleid'] 
neg_samples29 = df2.loc[:,list(samples)]

df_samples = df[df['disease']=='colon adenocarcinoma']
samples = df_samples[df_samples['pedaya']=='Yes, age < 30 years']['th_sampleid']
pos_samples30 = df2.loc[:,list(samples)]
samples = df_samples[df_samples['pedaya']=='No']['th_sampleid'] 
neg_samples30 = df2.loc[:,list(samples)]

In [15]:
pos_samples6 = pd.concat([pos_samples27.T, pos_samples28.T, pos_samples29.T, pos_samples30.T])
neg_samples6 = pd.concat([neg_samples27.T, neg_samples28.T, neg_samples29.T, neg_samples30.T])

In [229]:
write_n_most_variable("ped_v_nonped_adenocarcinomas", pos_samples6, neg_samples6, 8000)

<ipython-input-219-60ddce655a62>:9: RuntimeWarning: divide by zero encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -
<ipython-input-219-60ddce655a62>:9: RuntimeWarning: invalid value encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -


In [16]:
samples1 = df[df['disease']=='lung squamous cell carcinoma']['th_sampleid'] 
g1_samples = df2.loc[:,list(samples1)]

samples2 = df[df['disease']=='kidney clear cell carcinoma']['th_sampleid'] 
g2_samples = df2.loc[:,list(samples2)]

samples3 = df[df['disease']=='thyroid carcinoma']['th_sampleid'] 
g3_samples = df2.loc[:,list(samples3)]

samples4 = df[df['disease']=='head & neck squamous cell carcinoma']['th_sampleid'] 
g4_samples = df2.loc[:,list(samples4)]

samples5 = df[df['disease']=='kidney papillary cell carcinoma']['th_sampleid'] 
g5_samples = df2.loc[:,list(samples5)]

samples6 = df[df['disease']=='bladder urothelial carcinoma']['th_sampleid'] 
g6_samples = df2.loc[:,list(samples6)]

samples7 = df[df['disease']=='lung adenocarcinoma']['th_sampleid'] 
g7_samples = df2.loc[:,list(samples7)]

samples8 = df[df['disease']=='stomach adenocarcinoma']['th_sampleid'] 
g8_samples = df2.loc[:,list(samples8)]

samples9 = df[df['disease']=='prostate adenocarcinoma']['th_sampleid'] 
g9_samples = df2.loc[:,list(samples9)]

samples10 = df[df['disease']=='colon adenocarcinoma']['th_sampleid'] 
g10_samples = df2.loc[:,list(samples10)]

In [17]:
pos_samples7 = pd.concat([g1_samples.T, g2_samples.T, g3_samples.T, g4_samples.T, g5_samples.T, g6_samples.T])
neg_samples7 = pd.concat([g7_samples.T, g8_samples.T, g9_samples.T, g10_samples.T])

In [233]:
write_n_most_variable("carcinomas_v_adenocarcinomas", pos_samples7, neg_samples7, 8000)

<ipython-input-219-60ddce655a62>:9: RuntimeWarning: divide by zero encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -
<ipython-input-219-60ddce655a62>:9: RuntimeWarning: invalid value encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -


Writing feature-REDUCED datasets (down to the most variable 850 features)

In [107]:
# experiment 1
write_n_most_variable("male_v_female_carcinomas_reduced", pos_samples3, neg_samples3, 1200)

<ipython-input-95-f240ea54e7fd>:9: RuntimeWarning: divide by zero encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -
<ipython-input-95-f240ea54e7fd>:9: RuntimeWarning: invalid value encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -


In [101]:
df_pos1 = pd.read_csv('male_v_female_carcinomas_reduced_pos_samples.tsv', sep='\t', header=0)
df_pos1 = df_pos1.set_index(df_pos1.iloc[:,0])
df_pos1 = df_pos1.iloc[:, 1:]

df_neg1 = pd.read_csv('male_v_female_carcinomas_reduced_neg_samples.tsv', sep='\t', header=0)
df_neg1 = df_neg1.set_index(df_neg1.iloc[:,0])
df_neg1 = df_neg1.iloc[:, 1:]

In [102]:
df_pos1

,120,249,258,332,459,554,633,684,748,813,...,33260,33264,33265,33266,33267,33268,33269,33321,33344,33370
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
TCGA-18-3407-01,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.042784,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,1.459497,0.00000,0.056724,0.000000
TCGA-18-3409-01,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,1.238867,0.00000,0.124460,0.000000
TCGA-18-3410-01,0.0,0.028710,0.028710,0.0,0.000000,0.176446,0.0,0.495795,0.895387,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.433055,0.00000,0.084203,0.056724
TCGA-18-3412-01,0.0,0.014498,0.014498,0.0,0.000000,0.000000,0.0,0.084203,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.056724,0.00000,0.000000,0.000000
TCGA-18-3414-01,0.0,0.028710,0.028710,0.0,0.000000,0.084203,0.0,0.000000,0.000000,0.238915,...,0.0,0.0,0.0,0.0,0.000000,0.0,2.372941,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZF-AA4X-01,0.0,0.042784,0.042784,0.0,0.000000,0.028710,0.0,0.000000,0.000000,0.084203,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.176446,0.02871,0.000000,0.000000
TCGA-ZF-AA52-01,0.0,0.014498,0.014498,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.623024,0.00000,0.000000,0.000000
TCGA-ZF-AA53-01,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.097748,...,0.0,0.0,0.0,0.0,0.000000,0.0,1.400608,0.00000,0.056724,0.042784


In [108]:
# experiment 2
write_n_most_variable("male_v_female_adenocarcinomas_reduced", pos_samples4, neg_samples4, 1200)

<ipython-input-95-f240ea54e7fd>:9: RuntimeWarning: divide by zero encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -
<ipython-input-95-f240ea54e7fd>:9: RuntimeWarning: invalid value encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -


In [109]:
# experiment 3
write_n_most_variable("ped_v_nonped_carcinomas_reduced", pos_samples5, neg_samples5, 1200)

<ipython-input-95-f240ea54e7fd>:9: RuntimeWarning: divide by zero encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -
<ipython-input-95-f240ea54e7fd>:9: RuntimeWarning: invalid value encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -


In [105]:
# experiment 4 is trashed

In [110]:
# experiment 5
write_n_most_variable("carcinomas_v_adenocarcinomas_reduced", pos_samples7, neg_samples7, 1200)

<ipython-input-95-f240ea54e7fd>:9: RuntimeWarning: divide by zero encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -
<ipython-input-95-f240ea54e7fd>:9: RuntimeWarning: invalid value encountered in double_scalars
  relativeDifferences[i] = np.abs(pos_samples.iloc[:,i].mean() -
